In [1]:
# Preamble
import numpy as np
import glob
import matplotlib
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import os

In [2]:
scale_factor = 1000000 #MPa

# For the fault figure
cmax_f =   8000000 / scale_factor
cmin_f =  -8000000 / scale_factor
levels = np.linspace(cmin_f, cmax_f, 16)
colors_l = 'w'


xx_f, yy_f = np.loadtxt('vertx0.ts', delimiter=' ', usecols=(1, 2), unpack=True)
tri1, tri2, tri3 = np.loadtxt('triang0.ts', dtype=int, delimiter=' ', usecols=(0, 1, 2), unpack=True)
xx_f = xx_f/1000
yy_f = yy_f/1000
mu_s = 0.6

tria = []
for i in range(len(tri1)):
    tria.append([tri1[i]-1, tri2[i]-1, tri3[i]-1])

triangles = np.asarray(tria)
# read the files
list_files_m = glob.glob('output_offm2p5_gapm5p0/offm2p5_gapm5p0-faultreceivern*.dat')
list_files_p = glob.glob('output_offm2p5_gapp5p0/offm2p5_gapp5p0-faultreceivern*.dat')
list_files_m.sort()
list_files_p.sort()
n_files = len(list_files_m)
n_cols = 20
n_steps = 81

k = 0
count = 0
data_m = np.zeros((n_files,n_steps,n_cols), dtype=np.float64)
data_p = np.zeros((n_files,n_steps,n_cols), dtype=np.float64)
for f in range(len(list_files_m)):
    infile_m = open(list_files_m[f],'r')
    infile_p = open(list_files_p[f],'r')
    dat_m = infile_m.read()
    vals_m = dat_m.split("\n")
    dat_p = infile_p.read()
    vals_p = dat_p.split("\n")
    for i in range(n_steps):
        val_m = vals_m[i].split(" ")
        val_p = vals_p[i].split(" ")
        for j in range(n_cols):
            data_m[k][i][j] = float(val_m[j])
            data_p[k][i][j] = float(val_p[j])
    infile_m.close()
    infile_p.close()
    k += 1

xx_m = xx_f[triangles].mean(axis = 1)
yy_m = yy_f[triangles].mean(axis = 1)


In [3]:
# Load the virtual receiver coordinates (horizontal)
xx = np.loadtxt('xx_coord.dat', usecols=(0), unpack=True)
yy = np.loadtxt('yy_coord.dat', usecols=(0), unpack=True)

xx = xx / 100000
yy = yy / 100000

vet_trace = np.loadtxt('vettore_trace_4k_depth.dat', delimiter=' ', usecols=(0,1), unpack=True)
lag_trace = np.loadtxt('llaga_trace_overlap.dat', delimiter=' ', usecols=(0,1), unpack=True)
lag_trace_p5 = np.loadtxt('llaga_p5_4k.dat', delimiter=' ', usecols=(0,1), unpack=True)
lag_trace_m5 = np.loadtxt('llaga_m5_4k.dat', delimiter=' ', usecols=(0,1), unpack=True)

vet_trace = vet_trace / 100000
lag_trace = lag_trace / 100000
lag_trace_m5 = lag_trace_m5 / 100000
lag_trace_p5 = lag_trace_p5 / 100000

# Strike vector
stk_v = [-0.43051,   0.90259,   0.00000];
stk_v = np.array(stk_v)
a = [349584,  4.74132e+6,  -8500];
b = [354640,  4.74373e+6,  -500];
a = np.array(a)
b = np.array(b)
# Dip vector
dip_v = (a-b)/np.linalg.norm(a-b);

# Normal vector to the fault
n_v = np.cross(stk_v,dip_v);
n_v = n_v/np.linalg.norm(n_v);
n = n_v

# Static coefficient
mu_s = 0.6

# Rotation matrix
matrix = np.array([dip_v,stk_v,n_v])

# List of files
snaps_list = glob.glob('output_h_plus/lines_00*.dat')
snaps_list.sort()


# Read a given receiver and check the CFF time series at that receiver location
rec_id_fault = 19*4 + 15
rec_id_fault_2 = 19*6 + 7
path = 'output_offm2p5_gapm5p0/offm2p5_gapm5p0-faultreceiver-'
file_name = path + str(rec_id_fault).zfill(5) + '-00000.dat'
with open(file_name) as fin:
    lines = fin.readlines()
x_rec_f = float(lines[2].split("       ")[1])/100000
y_rec_f = float(lines[3].split("       ")[1])/100000
z_rec_f = float(lines[4].split("      ")[1])/1000
print("rec fault coords: ",x_rec_f,y_rec_f,z_rec_f)

path = 'output_offm2p5_gapm5p0/offm2p5_gapm5p0-faultreceiver-'
file_name = path + str(rec_id_fault_2).zfill(5) + '-00000.dat'
with open(file_name) as fin:
    lines = fin.readlines()
x_rec_f_2 = float(lines[2].split("       ")[1])/100000
y_rec_f_2 = float(lines[3].split("       ")[1])/100000
z_rec_f_2 = float(lines[4].split("      ")[1])/1000
print("rec fault coords: ",x_rec_f_2,y_rec_f_2,z_rec_f)



# Horizontal dimension
nx = 40;
ny = 55;

# Dimension vertical profile
nstk = 31;
nz = 30;

stk = np.linspace(0,500*(nstk-1),31)
zz = np.linspace(-500*(nz-1) + 500,500,30)

stk = stk / 1000;
zz = zz / 1000;

dt_stamp = 0.1   # sampling of output files
delay_samp = 6   # lines of header

# FOR THE RECEIVER PLOT

# Read a given receiver and check the CFF time series at that receiver location
rec_id = 31*6 + 18
path = 'output_v_plus_vet/offm2p5_gapm5p0-receiver-'
file_name = path + str(rec_id).zfill(5) + '-00000.dat'
with open(file_name) as fin:
    lines = fin.readlines()
path = 'output_v_plus_vet_f/offm2p5_gapp5p0-receiver-'
file_name = path + str(rec_id).zfill(5) + '-00000.dat'
with open(file_name) as fin:
    lines_2 = fin.readlines()
x_rec = float(lines[2].split("       ")[1])/100000
y_rec = float(lines[3].split("       ")[1])/100000
z_rec = float(lines[4].split("       ")[1])/1000
print("rec coords: ",x_rec,y_rec,z_rec)
nt = len(lines)-5
time_rec = np.zeros((nt))
sxx_rec = np.zeros((nt))
syy_rec = np.zeros((nt))
szz_rec = np.zeros((nt))
sxy_rec = np.zeros((nt))
syz_rec = np.zeros((nt))
sxz_rec = np.zeros((nt))
u_rec = np.zeros((nt))
v_rec = np.zeros((nt))
w_rec = np.zeros((nt))
t1_rec = np.zeros((nt))
t2_rec = np.zeros((nt))
t3_rec = np.zeros((nt))
s1_rec = np.zeros((nt))
s2_rec = np.zeros((nt))
s3_rec = np.zeros((nt))
delta_tau_rec = np.zeros((nt))
delta_sig_rec = np.zeros((nt))
delta_cff_rec = np.zeros((nt))

time_rec_2 = np.zeros((nt))
sxx_rec_2 = np.zeros((nt))
syy_rec_2 = np.zeros((nt))
szz_rec_2 = np.zeros((nt))
sxy_rec_2 = np.zeros((nt))
syz_rec_2 = np.zeros((nt))
sxz_rec_2 = np.zeros((nt))
u_rec_2 = np.zeros((nt))
v_rec_2 = np.zeros((nt))
w_rec_2 = np.zeros((nt))
t1_rec_2 = np.zeros((nt))
t2_rec_2 = np.zeros((nt))
t3_rec_2 = np.zeros((nt))
s1_rec_2 = np.zeros((nt))
s2_rec_2 = np.zeros((nt))
s3_rec_2 = np.zeros((nt))
delta_tau_rec_2 = np.zeros((nt))
delta_sig_rec_2 = np.zeros((nt))
delta_cff_rec_2 = np.zeros((nt))

max_c = []
min_c = []
for i in range(nt):
    time_rec[i] = lines[i+5].split("  ")[1]
    sxx_rec[i] = lines[i+5].split("  ")[2]
    syy_rec[i] = lines[i+5].split("  ")[3]
    szz_rec[i] = lines[i+5].split("  ")[4]
    sxy_rec[i] = lines[i+5].split("  ")[5]
    syz_rec[i] = lines[i+5].split("  ")[6]
    sxz_rec[i] = lines[i+5].split("  ")[7]
    u_rec[i] = lines[i+5].split("  ")[8]
    v_rec[i] = lines[i+5].split("  ")[9]
    w_rec[i] = lines[i+5].split("  ")[10]
    sig_rec = np.array([[sxx_rec[i], sxy_rec[i], sxz_rec[i]], [sxy_rec[i], syy_rec[i], syz_rec[i]], [sxz_rec[i], syz_rec[i], szz_rec[i]]])
    t = sig_rec.dot(n)
    # Shear and normal stress
    tau = t - (np.dot(n,t))*n
    sigma = np.dot(n,t)*n
    # Rotated vectors
    taup = matrix.dot(tau)
    sigmap = matrix.dot(sigma)
    t1_rec[i] = taup[0]
    t2_rec[i] = taup[1]
    t3_rec[i] = taup[2]
    s1_rec[i] = sigmap[0]
    s2_rec[i] = sigmap[1]
    s3_rec[i] = sigmap[2]
    delta_tau_rec[i] = t1_rec[i]            
    #%multiply delta_sigma by -1 to change the seissol convention (negative = compression)
    delta_sig_rec[i] = -1.0*sigmap[2]
    delta_cff_rec[i] = delta_tau_rec[i] + mu_s*delta_sig_rec[i]

    time_rec_2[i] = lines_2[i+5].split("  ")[1]
    sxx_rec_2[i] = lines_2[i+5].split("  ")[2]
    syy_rec_2[i] = lines_2[i+5].split("  ")[3]
    szz_rec_2[i] = lines_2[i+5].split("  ")[4]
    sxy_rec_2[i] = lines_2[i+5].split("  ")[5]
    syz_rec_2[i] = lines_2[i+5].split("  ")[6]
    sxz_rec_2[i] = lines_2[i+5].split("  ")[7]
    u_rec_2[i] = lines_2[i+5].split("  ")[8]
    v_rec_2[i] = lines_2[i+5].split("  ")[9]
    w_rec_2[i] = lines_2[i+5].split("  ")[10]
    sig_rec = np.array([[sxx_rec_2[i], sxy_rec_2[i], sxz_rec_2[i]], [sxy_rec_2[i], syy_rec_2[i], syz_rec_2[i]], [sxz_rec_2[i], syz_rec_2[i], szz_rec_2[i]]])
    t = sig_rec.dot(n)
    # Shear and normal stress
    tau = t - (np.dot(n,t))*n
    sigma = np.dot(n,t)*n
    # Rotated vectors
    taup = matrix.dot(tau)
    sigmap = matrix.dot(sigma)
    t1_rec_2[i] = taup[0]
    t2_rec_2[i] = taup[1]
    t3_rec_2[i] = taup[2]
    s1_rec_2[i] = sigmap[0]
    s2_rec_2[i] = sigmap[1]
    s3_rec_2[i] = sigmap[2]
    delta_tau_rec_2[i] = t1_rec_2[i]            
    #%multiply delta_sigma by -1 to change the seissol convention (negative = compression)
    delta_sig_rec_2[i] = -1.0*sigmap[2]
    delta_cff_rec_2[i] = delta_tau_rec_2[i] + mu_s*delta_sig_rec_2[i]

max_cff_rec = np.abs(np.max(delta_cff_rec_2)/1000000)
min_cff_rec = np.abs(np.min(delta_cff_rec_2)/1000000)
abs_cff_rec = np.max([max_cff_rec, min_cff_rec])
max_cff_rec = abs_cff_rec
min_cff_rec = -1.*abs_cff_rec

counter = 0
T_rec_m = []
P_rec_m = []
T_rec_p = []
P_rec_p = []
T_rec_m_2 = []
P_rec_m_2 = []
T_rec_p_2 = []
P_rec_p_2 = []
SRd_rec_m_2 = []
SRd_rec_p_2 = []
SRd_rec_m_2 = []
SRd_rec_p_2 = []

for it in snaps_list:
    time_stamp = ( float(it.split("/")[1][-8:-4]) - delay_samp ) * dt_stamp
    time_lim =  int(it.split("/")[1][-8:-4]) - delay_samp
    print(it)
    # Cut recording for the receiver plot
    ttime_rec = time_rec[0:time_lim]
    ddelta_cff_rec = delta_cff_rec[0:time_lim]
    ttime_rec_2 = time_rec_2[0:time_lim]
    ddelta_cff_rec_2 = delta_cff_rec_2[0:time_lim]
    time_stamp_title = "Time: " + format(0.2*counter, '.1f') + ' s'
    it_compare = 'output_h_plus_f/' + it.split("/")[1]
    out1 = 'horizontal_delta_' + str(counter).zfill(5) + '.png'

    # TRIANGLES DATA
    ##########################################
    col_Td = 4
    col_Pn = 5
    fileout = 'fault_' + str(counter).zfill(5) + '.png'
    T_d_m = []
    P_n_m = []
    T_d_p = []
    P_n_p = []
    SRd_p = []
    SRd_m = []
    for i in range(len(xx_f)):
        # T_d column 4
        # P_n column 5
        T_d_m.append(data_m[i][counter][col_Td])
        P_n_m.append(data_m[i][counter][col_Pn])
        T_d_p.append(data_p[i][counter][col_Td])
        P_n_p.append(data_p[i][counter][col_Pn])
        SRd_p.append(data_p[i][counter][2])
        SRd_m.append(data_m[i][counter][2])
    # Receiver on fault #1
    T_rec_m.append(data_m[rec_id_fault][counter][col_Td])
    P_rec_m.append(data_m[rec_id_fault][counter][col_Pn])    
    T_rec_p.append(data_p[rec_id_fault][counter][col_Td])
    P_rec_p.append(data_p[rec_id_fault][counter][col_Pn])    
    T_rec_m_arr = np.asarray(T_rec_m)
    P_rec_m_arr = np.asarray(P_rec_m)
    T_rec_p_arr = np.asarray(T_rec_p)
    P_rec_p_arr = np.asarray(P_rec_p)
    tt = np.linspace(0,(counter*0.2),counter+1)
    D_cff_rec_m = T_rec_m_arr - mu_s * P_rec_m_arr
    D_cff_rec_p = T_rec_p_arr - mu_s * P_rec_p_arr    
    # Receiver on fault #2
    T_rec_m_2.append(data_m[rec_id_fault_2][counter][col_Td])
    P_rec_m_2.append(data_m[rec_id_fault_2][counter][col_Pn])    
    T_rec_p_2.append(data_p[rec_id_fault_2][counter][col_Td])
    P_rec_p_2.append(data_p[rec_id_fault_2][counter][col_Pn])    
    T_rec_m_2_arr = np.asarray(T_rec_m_2)
    P_rec_m_2_arr = np.asarray(P_rec_m_2)
    T_rec_p_2_arr = np.asarray(T_rec_p_2)
    P_rec_p_2_arr = np.asarray(P_rec_p_2)
    D_cff_rec_m_2 = T_rec_m_2_arr - mu_s * P_rec_m_2_arr
    D_cff_rec_p_2 = T_rec_p_2_arr - mu_s * P_rec_p_2_arr    
    SRd_m_rec.append(data_m[rec_id_fault][counter][2])
    SRd_p_rec.append(data_p[rec_id_fault][counter][2])
    SRd_m_rec_2.append(data_m[rec_id_fault_2][counter][2])
    SRd_p_rec_2.append(data_p[rec_id_fault_2][counter][2])

    SRd_p_rec_2_arr = np.asarray(SRd_p_rec_2)
    SRd_m_rec_2_arr = np.asarray(SRd_m_rec_2)
    SRd_p_rec_arr = np.asarray(SRd_p_rec)
    SRd_m_rec_arr = np.asarray(SRd_m_rec)
    
    
    T_d_m = np.asarray(T_d_m)
    P_n_m = np.asarray(P_n_m) * -1. # seissol convention test
    T_d_p = np.asarray(T_d_p)
    P_n_p = np.asarray(P_n_p) * -1.
    D_cff_m = T_d_m - mu_s * P_n_m
    D_cff_p = T_d_p - mu_s * P_n_p
    D_cff_m = D_cff_m / scale_factor
    D_cff_p = D_cff_p / scale_factor
    
    #max_c_m.append(max(D_cff_m))
    #min_c_m.append(min(D_cff_m))
    #max_c_p.append(max(D_cff_p))
    #min_c_p.append(min(D_cff_p))
    #zfaces_m = SRd_m[triangles].mean(axis = 1) #D_cff_m[triangles].mean(axis = 1)
    #zfaces_p = #D_cff_p[triangles].mean(axis = 1)
    ########################################

    counter = counter + 1

    #Load the data horizontal
    time_hm, sxx_hm, syy_hm, szz_hm, sxy_hm, syz_hm, sxz_hm, u_hm, v_hm, w_hm = np.loadtxt(it_compare, delimiter=' ', usecols=(0,1,2,3,4,5,6,7,8,9), unpack=True)

    #Load the data horizontal
    time_hp, sxx_hp, syy_hp, szz_hp, sxy_hp, syz_hp, sxz_hp, u_hp, v_hp, w_hp = np.loadtxt(it, delimiter=' ', usecols=(0,1,2,3,4,5,6,7,8,9), unpack=True)

    # Save space for variables
    t1_hm = np.zeros((ny,nx))
    t2_hm = np.zeros((ny,nx))
    t3_hm = np.zeros((ny,nx))
    s1_hm = np.zeros((ny,nx))
    s2_hm = np.zeros((ny,nx))
    s3_hm = np.zeros((ny,nx))
    delta_tau_hm = np.zeros((ny,nx))
    delta_sig_hm = np.zeros((ny,nx))
    delta_cff_hm = np.zeros((ny,nx))
    uhm = np.zeros((ny,nx))
    vhm = np.zeros((ny,nx))
    whm = np.zeros((ny,nx))

    # Save space for variables
    t1_hp = np.zeros((ny,nx))
    t2_hp = np.zeros((ny,nx))
    t3_hp = np.zeros((ny,nx))
    s1_hp = np.zeros((ny,nx))
    s2_hp = np.zeros((ny,nx))
    s3_hp = np.zeros((ny,nx))
    delta_tau_hp = np.zeros((ny,nx))
    delta_sig_hp = np.zeros((ny,nx))
    delta_cff_hp = np.zeros((ny,nx))
    uhp = np.zeros((ny,nx))
    vhp = np.zeros((ny,nx))
    whp = np.zeros((ny,nx))

    
    #invert order
    ndata_hm = len(time_hm);
    k = ndata_hm-1;
    n = n_v;
    ii = nx;
    for i in range(nx):
        jj = ny;
        for j in range(ny):
            sig_hm = np.array([[sxx_hm[k], sxy_hm[k], sxz_hm[k]], [sxy_hm[k], syy_hm[k], syz_hm[k]], [sxz_hm[k], syz_hm[k], szz_hm[k]]])
            t = sig_hm.dot(n)
            # Shear and normal stress
            tau = t - (np.dot(n,t))*n
            sigma = np.dot(n,t)*n
            # Rotated vectors
            taup = matrix.dot(tau)
            sigmap = matrix.dot(sigma)
            t1_hm[j][i] = taup[0]
            t2_hm[j][i] = taup[1]
            t3_hm[j][i] = taup[2]
            s1_hm[j][i] = sigmap[0]
            s2_hm[j][i] = sigmap[1]
            s3_hm[j][i] = sigmap[2]
            delta_tau_hm[j][i] = t1_hm[j][i]            
            #%multiply delta_sigma by -1 to change the seissol convention (negative = compression)
            delta_sig_hm[j][i] = -1.0*sigmap[2]
            delta_cff_hm[j][i] = delta_tau_hm[j][i] + mu_s*delta_sig_hm[j][i]
            uhm[j][i] = u_hm[k]
            vhm[j][i] = v_hm[k]
            whm[j][i] = w_hm[k]
            jj = jj-1
            k=k-1
    delta_cff_hm = delta_cff_hm / scale_factor

    #invert order
    ndata_hp = len(time_hp);
    k = ndata_hp-1;
    n = n_v;
    ii = nx;
    for i in range(nx):
        jj = ny;
        for j in range(ny):
            sig_hp = np.array([[sxx_hp[k], sxy_hp[k], sxz_hp[k]], [sxy_hp[k], syy_hp[k], syz_hp[k]], [sxz_hp[k], syz_hp[k], szz_hp[k]]])
            t = sig_hp.dot(n)
            # Shear and normal stress
            tau = t - (np.dot(n,t))*n
            sigma = np.dot(n,t)*n
            # Rotated vectors
            taup = matrix.dot(tau)
            sigmap = matrix.dot(sigma)
            t1_hp[j][i] = taup[0]
            t2_hp[j][i] = taup[1]
            t3_hp[j][i] = taup[2]
            s1_hp[j][i] = sigmap[0]
            s2_hp[j][i] = sigmap[1]
            s3_hp[j][i] = sigmap[2]
            delta_tau_hp[j][i] = t1_hp[j][i]            
            #%multiply delta_sigma by -1 to change the seissol convention (negative = compression)
            delta_sig_hp[j][i] = -1.0*sigmap[2]
            delta_cff_hp[j][i] = delta_tau_hp[j][i] + mu_s*delta_sig_hp[j][i]
            uhp[j][i] = u_hp[k]
            vhp[j][i] = v_hp[k]
            whp[j][i] = w_hp[k]
            jj = jj-1
            k=k-1
    delta_cff_hp = delta_cff_hp / scale_factor

    # Figures    hf, ha = plt.subplots(3,2)

    ninter = 5
  # for contour plots
    c_max_d =  8000000 / scale_factor
    c_min_d = -8000000 / scale_factor
    
    c_min_s = -5000000
    c_max_s =  5000000
    
    c_min_w = -0.37
    c_max_w =  0.32
    
    
    inter_d = np.linspace(c_min_d,c_max_d,ninter)
    inter_s = np.linspace(c_min_s,c_max_s,ninter)
    inter_w = np.linspace(c_min_w,c_max_w,ninter)

    cmap = matplotlib.cm.get_cmap('jet')
    
    # DELTA PLOTS
    fig1, axs1 = plt.subplots(2, 3, dpi=300, facecolor='white')
    fig1.suptitle(time_stamp_title, x=0.9)
    # delta tau
    # Delta CFF
    ax1 = axs1[0, 0]
    c = ax1.pcolor(xx, yy, delta_cff_hm, cmap=cmap, vmin=c_min_d, vmax=c_max_d)
    d = ax1.plot(vet_trace[0][:], vet_trace[1][:], linestyle='dashed', color='black', linewidth=0.3)
    e = ax1.plot(lag_trace_p5[0][:], lag_trace_p5[1][:], linestyle='dashed', color='black', linewidth=0.3)
    #ax1.scatter(x_rec, y_rec, 10, marker='o', color='black', facecolors='none')
    #f = ax3.contour(xx, yy, delta_cff_h, inter_d, colors='white', linewidth=0.4)
    ax1.set_position([0.1,0.6, 0.3, 0.34])
    ax1.set_title(r'$\Delta$CFF', fontsize='large')
    ax1.axis('equal')
    ax1.set_aspect('equal', 'box')
    ax1.set_ylabel('Latitude')
    ax1.set_xlabel('Longitude')
    
    ax2 = axs1[0, 1]
    c = ax2.pcolor(xx, yy, delta_cff_hp, cmap=cmap, vmin=c_min_d, vmax=c_max_d)
    d = ax2.plot(vet_trace[0][:], vet_trace[1][:], linestyle='dashed', color='black', linewidth=0.3)
    e = ax2.plot(lag_trace_m5[0][:], lag_trace_m5[1][:], linestyle='dashed', color='black', linewidth=0.3)
    #ax2.scatter(x_rec, y_rec, 10, marker='o', color='black', facecolors='none')
    #f = ax3.contour(xx, yy, delta_cff_h, inter_d, colors='white', linewidth=0.4)
    ax2.set_position([0.28,0.6, 0.3, 0.34])
    ax2.set_title(r'$\Delta$CFF', fontsize='large')
    ax2.set_yticklabels([])
    ax2.axis('equal')
    ax2.set_aspect('equal', 'box')
    ax2.set_xlabel('Longitude')

    #ax3.set_position([0.35,0.1, 0.35, 0.35])
    cbaxes = fig1.add_axes([0.53, 0.595, 0.02, 0.345]) 
    cb = fig1.colorbar(c, ax=ax1, cax=cbaxes)
    cb.set_label('Stress change (MPa)')

    ax3 = axs1[1, 0]
    hp = ax3.plot(tt, D_cff_rec_p, lw=1, color='tab:blue')
    hm = ax3.plot(tt, D_cff_rec_m, lw=1, color='tab:blue', linestyle='dashed')
    #just for the legend
    hp = ax3.plot(tt, 50e6+D_cff_rec_p, lw=2, color='black', label='Vet_on_footwall')
    hm = ax3.plot(tt, 50e6+D_cff_rec_m, lw=2, color='black', linestyle='dashed', label='Vet_on_hangingwall')
    #end of thing for the legend
    ax3.scatter(y_rec_f*100, z_rec_f, 20, edgecolor='black', color='blue', label='Shallow')
    ax3.set_position([0.16,0.25, 0.47, 0.15])
    ax3.set_xlim((0, np.max(time_rec)))
    #ax3.set_ylim((np.min(delta_cff_rec_2)*1.3, np.max(delta_cff_rec_2)*1.3))
    ax3.set_ylim((-500000, 500000))
    #ax3.set_xlabel('Time (s)')
    ax3.set_xticklabels([])
    ax3.set_ylabel(r'$\Delta$CFF (MPa)')
    ax3.grid()
    leg = ax3.legend(ncol=2, loc=[0.12, 1.01], prop={'size': 8})
    leg.get_frame().set_edgecolor('none')
    leg.get_frame().set_facecolor('none')

    ax6 = axs1[0, 2]
    hp = ax6.plot(tt, D_cff_rec_p_2, lw=1, color='tab:red')
    hm = ax6.plot(tt, D_cff_rec_m_2, lw=1, color='tab:red', linestyle='dashed')
    ax6.scatter(y_rec_f_2*100, z_rec_f_2, 20, edgecolor='black', color='red', label='Deep')
    ax6.set_position([0.16,0.05, 0.47, 0.15])
    ax6.set_xlim((0, np.max(time_rec)))
    #ax3.set_ylim((np.min(delta_cff_rec_2)*1.3, np.max(delta_cff_rec_2)*1.3))
    ax6.set_ylim((-500000, 500000))
    ax6.set_xlabel('Time (s)')
    ax6.set_ylabel(r'$\Delta$CFF (MPa)')
    ax6.grid()
    leg = ax6.legend(ncol=2, loc=[0.7, 1.01], prop={'size': 8})
    leg.get_frame().set_edgecolor('none')
    leg.get_frame().set_facecolor('none')

    
    cmap = matplotlib.cm.get_cmap('seismic')

    c_max_d =  250000 / scale_factor
    c_min_d = -250000 / scale_factor
    levels = np.linspace(c_min_d, c_max_d, 6)

    ax4 = axs1[1, 1]
    ax4.set_aspect('equal')
    ax4.set_position([0.75,0.2, 0.25, 0.25])
    tpc_m = ax4.tripcolor(xx_f, yy_f, triangles,
                    facecolors = zfaces_m,
                    cmap = cmap,
                    edgecolors ='k',
                    #shading = 'flat',
                    vmax = c_max_d,
                    vmin = c_min_d)
    #tc = ax4.tricontour(xx_m, yy_m, zfaces_m, levels=levels, linewidths=0.2, colors=colors_l)
    tcl = ax4.clabel(tc, colors='w', fontsize=3, inline=1, fmt = '%1.0f')
    ax4.scatter(y_rec_f*100, z_rec_f, 5, edgecolor='black', color='blue')
    ax4.scatter(y_rec_f_2*100, z_rec_f_2, 5, edgecolor='black', color='red')
    ax4.set_ylabel('Depth (km)')

    cbaxes = fig1.add_axes([0.76, 0.11, 0.23, 0.03]) 
    cb = fig1.colorbar(tpc_m, ax=ax4, cax=cbaxes, orientation='horizontal')
    cb.set_label('Stress change (MPa)')
    
    c_max_d =   250000 / scale_factor
    c_min_d =  -250000 / scale_factor
    levels = np.linspace(c_min_d, c_max_d, 6)

    ax5 = axs1[1, 2]
    ax5.set_aspect('equal')
    ax5.set_position([0.75,0.68, 0.25, 0.25])
    #ax5.set_xticklabels([])
    tpc_p = ax5.tripcolor(xx_f, yy_f, triangles,
                    facecolors = zfaces_p,
                    cmap = cmap,
                    edgecolors ='k',
                    #shading = 'flat',
                    vmax = c_max_d,
                    vmin = c_min_d)
    #tc = ax5.tricontour(xx_m, yy_m, zfaces_p, levels=levels, linewidths=0.2, colors=colors_l)
    tcl = ax5.clabel(tc, colors='w', fontsize=5, inline=1, fmt = '%1.0f')
    ax5.scatter(y_rec_f*100, z_rec_f, 5, edgecolor='black', color='blue')
    ax5.scatter(y_rec_f_2*100, z_rec_f_2, 5, edgecolor='black', color='red')
    ax5.set_ylabel('Depth (km)')
    
    cbaxes = fig1.add_axes([0.76, 0.59, 0.23, 0.03]) 
    cb = fig1.colorbar(tpc_p, ax=ax5, cax=cbaxes, orientation='horizontal')
    cb.set_label('Stress change (MPa)')

    

    #cntr2 = ax4.tricontourf(xx_m, yy_m, zfaces, levels=14, cmap="RdBu_r")
    
    fig1.savefig(out1)




FileNotFoundError: [Errno 2] No such file or directory: 'output_offm2p5_gapm5p0/offm2p5_gapm5p0-faultreceiver-00091-00000.dat'

In [4]:
print(y_rec_f_2,z_rec_f_2,len(triangles),len(xx_f),len(zfaces_m))

NameError: name 'y_rec_f_2' is not defined